# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [65]:
import numpy as np
import xarray as xr
import cmomy 

nsamp = 100
energy = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

position = xr.DataArray(np.random.rand(nsamp, 3), dims=['samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nsamp), dims=['samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

In [67]:
ce

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([5.06193744e+01, 4.85667174e-01, 1.00086947e-01, 1.79898933e-03])
Dimensions without coordinates: mom_0

In [68]:
cp

<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 5.06193744e+01,  5.03033065e-01,  8.54233564e-02,
        -1.66602612e-03],
       [ 5.06193744e+01,  5.35356680e-01,  7.57106740e-02,
        -1.81183817e-03],
       [ 5.06193744e+01,  5.03974090e-01,  7.76862469e-02,
         2.88619887e-03]])
Dimensions without coordinates: dim, mom_0

## Basic attribues

Notice that there are three `shape` parameters associated with a `CentraMoments` object:
* `mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* `val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vectory data, 
* `shape`: total shape of wrapped moments data.

In [69]:
ce.mom_shape, ce.val_shape, ce.shape

((4,), (), (4,))

In [70]:
cp.mom_shape, cp.val_shape, cp.shape

((4,), (3,), (3, 4))

To access the underlying data, use the `values` attribute.  `values[...,0]`  is the total weight, `values[...,1]` is the average, and `values[...,k>1]` is the 'kth' central moment.

In [71]:
ce.values

<xarray.DataArray (mom_0: 4)>
array([5.06193744e+01, 4.85667174e-01, 1.00086947e-01, 1.79898933e-03])
Dimensions without coordinates: mom_0

To access all the central moments (zeroth and first included), use

In [72]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.        , 0.10008695, 0.00179899])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments. 

In [73]:
#<x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.        , 0.48566717, 0.33595955, 0.2621814 ])
Dimensions without coordinates: mom_0

Additionally, there are `DataArray` like attributes

In [76]:
ce.coords

Coordinates:
    *empty*

In [77]:
ce.attrs

{}

In [78]:
ce.sizes

Frozen({'mom_0': 4})

In [79]:
ce.dims

('mom_0',)

## Manipulating momements

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [85]:
nsamp = 100
nrec = 10
energy = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])
position = xr.DataArray(np.random.rand(nrec, nsamp, 3), dims=['rec','samp','dim'])

# weight assicated with each sample and each record
w = xr.DataArray(np.random.rand(nrec, nsamp), dims=['rec','samp'])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim='samp', mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim='samp', mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the `block` method.

In [86]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 4.92161375e+01,  4.96286586e-01,  7.63320152e-02,
         9.83313299e-04],
       [ 4.79346462e+01,  5.56126933e-01,  8.34660453e-02,
        -4.32337037e-03],
       [ 4.91375958e+01,  4.79650517e-01,  8.49124095e-02,
         4.94994872e-03],
       [ 4.71946550e+01,  5.30573998e-01,  7.80557249e-02,
        -2.78788725e-03],
       [ 4.97849156e+01,  4.80030444e-01,  6.81438391e-02,
        -3.32591418e-03],
       [ 4.63342819e+01,  5.53904787e-01,  7.04727344e-02,
        -4.54946749e-03],
       [ 4.61538031e+01,  5.54482701e-01,  6.45338036e-02,
        -7.88042115e-03],
       [ 5.05009061e+01,  4.50224567e-01,  9.33560336e-02,
         7.92266385e-03],
       [ 4.78969396e+01,  4.68702994e-01,  7.94262872e-02,
         1.51824705e-03],
       [ 5.20753683e+01,  5.39161500e-01,  8.15416852e-02,
        -9.02836313e-04]])
Dimensions without coordinates: rec, mom_0

In [87]:
ce.block(block_size=2, dim='rec')

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 9.71507837e+01,  5.25812090e-01,  8.07470397e-02,
        -1.31420105e-03],
       [ 9.63322508e+01,  5.04598715e-01,  8.22012505e-02,
         8.97961008e-04],
       [ 9.61191976e+01,  5.15641589e-01,  7.06290799e-02,
        -3.78324675e-03],
       [ 9.66547092e+01,  5.00009098e-01,  8.23050099e-02,
        -1.85994071e-03],
       [ 9.99723079e+01,  5.05404684e-01,  8.17671258e-02,
         3.65051792e-04]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data.  We produce a 20 new samples taken from the original (averaged) data.

In [88]:
ce.resample_and_reduce(nrep=20, dim='rec')

<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 4.86809319e+02,  4.84865517e-01,  8.16540290e-02,
         1.71571392e-03],
       [ 4.84164094e+02,  4.95904708e-01,  8.10990632e-02,
         4.63394099e-04],
       [ 4.92570148e+02,  5.09335333e-01,  8.13193551e-02,
        -1.31125465e-03],
       [ 4.93580894e+02,  5.17440571e-01,  8.18938836e-02,
        -4.66120637e-04],
       [ 4.79472364e+02,  5.25953071e-01,  7.78516958e-02,
        -2.25491157e-03],
       [ 4.92839479e+02,  4.93492277e-01,  8.23313494e-02,
         1.15023788e-03],
       [ 4.91713170e+02,  5.17139133e-01,  7.66862981e-02,
        -1.70544148e-03],
       [ 4.89467882e+02,  5.11676861e-01,  7.97397616e-02,
        -9.58804016e-04],
       [ 4.87476311e+02,  4.96871488e-01,  8.11355037e-02,
        -1.72928796e-04],
       [ 4.95010045e+02,  5.13620329e-01,  8.17407522e-02,
        -5.24705066e-04],
       [ 4.86539478e+02,  5.10861206e-01,  8.05717829e-02,
       

This is differnt than the usual 'resample values'.  This is also available if the original data is available.

In [91]:
a = energy.stack(c=['rec','samp'])

In [92]:
b = w.stack(c=['rec','samp'])

In [95]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(energy.stack(c=['rec','samp']), w=w.stack(c=['rec','samp']), dim='c', nrep=20, mom=3)

We can also reduce our original data across all the records

In [102]:
ce.reduce(dim='rec')

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([ 4.86229249e+02,  5.10273599e-01,  7.96405757e-02, -1.17644129e-03])
Dimensions without coordinates: mom_0